In [1]:
import os
import pandas as pd

In [2]:
# does not yet extract anything from the env_triad_pvs_vs_*.SSSOM.tsv files
#   because they don't use nmdc or mixs CURIes

In [3]:
# Define the directory containing the TSV files
directory = "../lexmatch-output"

output_tsv = "lexmatch-combined.tsv"

for_lexmatch_txt = '../for-lexmatch.txt'

env_triad_pvs_sheet_tsv = "env_triad_pvs_sheet.tsv"

In [4]:

env_triad_pvs_frame = pd.read_csv(env_triad_pvs_sheet_tsv, sep='\t', skiprows=[1])


In [5]:
env_triad_pvs_curies = list(env_triad_pvs_frame['class_uri'])

In [6]:
# Initialize an empty list to collect DataFrames
dataframes = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".tsv"):
        print(filename)
        file_path = os.path.join(directory, filename)

        # Read the file while skipping rows that start with #
        with open(file_path, 'r') as f:
            lines = [line for line in f if not line.startswith('#')]

        # Convert the filtered lines to a DataFrame
        if lines and len(lines) > 1:
            print(len(lines))
            from io import StringIO

            df = pd.read_csv(StringIO(''.join(lines)), sep='\t')
            dataframes.append(df)


nmdc_mixs_vs_bero.SSSOM.tsv
210
nmdc_mixs_vs_obiws.SSSOM.tsv
520
env_triad_pvs_vs_biolink.SSSOM.tsv
149
nmdc_nmdc_vs_ecocore.SSSOM.tsv
1422
env_triad_pvs_vs_sio.SSSOM.tsv
9
nmdc_nmdc_vs_pato.SSSOM.tsv
1482
nmdc_mixs_vs_ecso.SSSOM.tsv
34
env_triad_pvs_vs_pato.SSSOM.tsv
5223
nmdc_mixs_vs_omrse.SSSOM.tsv
539
env_triad_pvs_vs_omo.SSSOM.tsv
3
nmdc_mixs_vs_pw.SSSOM.tsv
547
env_triad_pvs_vs_obi.SSSOM.tsv
8
nmdc_nmdc_vs_omo.SSSOM.tsv
1351
nmdc_mixs_vs_mmo.SSSOM.tsv
831
nmdc_mixs_vs_peco.SSSOM.tsv
1141
nmdc_nmdc_vs_sio.SSSOM.tsv
1464
nmdc_mixs_vs_miapa.SSSOM.tsv
533
nmdc_nmdc_vs_chmo.SSSOM.tsv
1499
nmdc_mixs_vs_phipo.SSSOM.tsv
2225
nmdc_nmdc_vs_envthes.SSSOM.tsv
1437
nmdc_mixs_vs_cio.SSSOM.tsv
511
nmdc_mixs_vs_cdao.SSSOM.tsv
513
env_triad_pvs_vs_ecosim.SSSOM.tsv
110
nmdc_mixs_vs_ido.SSSOM.tsv
8
nmdc_nmdc_vs_biolink.SSSOM.tsv
1525
nmdc_nmdc_vs_envo.SSSOM.tsv
1708
nmdc_mixs_vs_bto.SSSOM.tsv
633
nmdc_nmdc_vs_ecto.SSSOM.tsv
1601
nmdc_mixs_vs_omit.SSSOM.tsv
47
env_triad_pvs_vs_goldterms.SSSOM.tsv
16

In [7]:
# Concatenate all DataFrames into a single DataFrame
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("Aggregated DataFrame created successfully!")
else:
    print("No data found.")


Aggregated DataFrame created successfully!


In [8]:
combined_df.shape

(731312, 11)

In [9]:
# Remove duplicate rows and force a new copy
unique_df = combined_df.drop_duplicates().copy()


In [10]:
unique_df.shape

(640007, 11)

In [11]:
non_reflexive_by_term_df = unique_df[unique_df['subject_id'] != unique_df['object_id']].copy()

In [12]:
non_reflexive_by_term_df.shape

(639152, 11)

In [13]:
# Step 1: Split prefixes first
non_reflexive_by_term_df['subject_prefix'] = non_reflexive_by_term_df['subject_id'].str.split(':').str[0]
non_reflexive_by_term_df['object_prefix'] = non_reflexive_by_term_df['object_id'].str.split(':').str[0]

In [14]:
# List of values to filter out
privileged_prefixes = ['mixs', 'nmdc']

# Columns to check
prefix_columns = ['subject_prefix', 'object_prefix']

In [15]:
non_reflexive_by_prefix_df = non_reflexive_by_term_df[non_reflexive_by_term_df['subject_prefix'] != non_reflexive_by_term_df['object_prefix']].copy()

In [16]:
non_reflexive_by_prefix_df.shape

(10750, 13)

In [17]:
# Filter rows where neither 'subject_prefix' nor 'object_prefix' are in privileged_prefixes
enriched_for_pvs_frame = non_reflexive_by_prefix_df[
    ~non_reflexive_by_prefix_df[prefix_columns].isin(privileged_prefixes).any(axis=1)
]


In [18]:
enriched_for_pvs_frame.shape

(8194, 13)

In [19]:
enriched_for_pvs_frame

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string,subject_prefix,object_prefix
774,biolink:broad_match,broad match,skos:closeMatch,skos:broadMatch,NaN,semapv:LexicalMatching,oaklib,0.5,rdf:ID,skos:exactMatch-INVERSE,biolink:broad_match,biolink,skos
784,biolink:close_match,close match,skos:closeMatch,skos:closeMatch,NaN,semapv:LexicalMatching,oaklib,0.5,rdf:ID,skos:exactMatch-INVERSE,biolink:close_match,biolink,skos
819,biolink:exact_match,exact match,skos:closeMatch,skos:exactMatch,NaN,semapv:LexicalMatching,oaklib,0.5,rdf:ID,skos:exactMatch-INVERSE,biolink:exact_match,biolink,skos
836,biolink:license,license,skos:closeMatch,dcterms:license,NaN,semapv:LexicalMatching,oaklib,0.5,rdf:ID,skos:exactMatch-INVERSE,biolink:license,biolink,dcterms
839,biolink:name,name,skos:closeMatch,dcterms:title,NaN,semapv:LexicalMatching,oaklib,0.5,rdf:ID,skos:narrowMatch-INVERSE,biolink:name,biolink,dcterms
...,...,...,...,...,...,...,...,...,...,...,...,...,...
727569,PO:0025060,lamina,skos:closeMatch,UBERON:0022303,nervous system cell part layer,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasBroadSynonym,lamina,PO,UBERON
729293,CHEMINF:000340,software module,skos:closeMatch,IAO:0000592,software module,semapv:LexicalMatching,oaklib,0.5,rdfs:label,rdfs:label,module software,CHEMINF,IAO
729294,CHEMINF:000341,software method,skos:closeMatch,IAO:0000591,software method,semapv:LexicalMatching,oaklib,0.5,rdfs:label,rdfs:label,method software,CHEMINF,IAO
729295,CHEMINF:000342,software library,skos:closeMatch,IAO:0000593,software library,semapv:LexicalMatching,oaklib,0.5,rdfs:label,rdfs:label,library software,CHEMINF,IAO


In [20]:
# Identify all subject_* and object_* columns dynamically
subject_columns = [col for col in non_reflexive_by_term_df.columns if col.startswith('subject_')]
object_columns = [col for col in non_reflexive_by_term_df.columns if col.startswith('object_')]

In [21]:
subject_columns

['subject_id', 'subject_label', 'subject_match_field', 'subject_prefix']

In [22]:
object_columns

['object_id', 'object_label', 'object_match_field', 'object_prefix']

In [23]:
mixs_nmdc_subj_frame = non_reflexive_by_term_df[non_reflexive_by_term_df["subject_prefix"].isin(['mixs', 'nmdc'])]

In [24]:
mixs_nmdc_obj_frame = non_reflexive_by_term_df[non_reflexive_by_term_df["object_prefix"].isin(['mixs', 'nmdc'])]

In [25]:
# Create a copy with swapped columns
reversed_obj_frame = mixs_nmdc_obj_frame.rename(columns={
    'subject_id': 'object_id_temp',
    'subject_label': 'object_label_temp',
    'subject_prefix': 'object_prefix_temp',
    'subject_match_field': 'object_match_field_temp',

    'object_id': 'subject_id',
    'object_label': 'subject_label',
    'object_prefix': 'subject_prefix',
    'object_match_field': 'subject_match_field'
})

# # Complete the swap by renaming the temporary columns
reversed_obj_frame = reversed_obj_frame.rename(columns={
    'object_id_temp': 'object_id',
    'object_label_temp': 'object_label',
    'object_prefix_temp': 'object_prefix',
    'object_match_field_temp': 'object_match_field'
})

In [26]:
# Create a copy with swapped columns
reversed_enriched_for_pvs_frame = enriched_for_pvs_frame.rename(columns={
    'subject_id': 'object_id_temp',
    'subject_label': 'object_label_temp',
    'subject_prefix': 'object_prefix_temp',
    'subject_match_field': 'object_match_field_temp',

    'object_id': 'subject_id',
    'object_label': 'subject_label',
    'object_prefix': 'subject_prefix',
    'object_match_field': 'subject_match_field'
})

# # Complete the swap by renaming the temporary columns
reversed_enriched_for_pvs_frame = reversed_enriched_for_pvs_frame.rename(columns={
    'object_id_temp': 'object_id',
    'object_label_temp': 'object_label',
    'object_prefix_temp': 'object_prefix',
    'object_match_field_temp': 'object_match_field'
})

In [27]:
enriched_for_pvs_frame = pd.concat([enriched_for_pvs_frame,reversed_enriched_for_pvs_frame], axis=0, ignore_index=True)

In [28]:
enriched_for_pvs_frame = enriched_for_pvs_frame[enriched_for_pvs_frame['subject_id'].isin(env_triad_pvs_curies)]

In [29]:
enriched_for_pvs_frame

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification,mapping_tool,confidence,subject_match_field,object_match_field,match_string,subject_prefix,object_prefix
8,ENVO:00000167,bar,skos:closeMatch,SIO:000519,bar,semapv:LexicalMatching,oaklib,0.5,rdfs:label,rdfs:label,bar,ENVO,SIO
9,ENVO:00002000,slope,skos:closeMatch,SIO:001184,slope,semapv:LexicalMatching,oaklib,0.5,rdfs:label,rdfs:label,slope,ENVO,SIO
39,ENVO:00000174,isthmus,skos:closeMatch,UBERON:0003052,midbrain-hindbrain boundary,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasRelatedSynonym,isthmus,ENVO,UBERON
78,PO:0025356,bulb,skos:closeMatch,UBERON:0001896,medulla oblongata,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasBroadSynonym,bulb,PO,UBERON
162,ENVO:00000011,garden,skos:closeMatch,GOLDTERMS:5774,Environmental > Terrestrial > Soil > Garden,semapv:LexicalMatching,oaklib,0.5,rdf:ID,skos:exactMatch,envo:00000011,ENVO,GOLDTERMS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16009,PO:0009001,fruit,skos:closeMatch,FOODON:03304644,"fruit (whole, raw)",semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasExactSynonym,fruit,PO,FOODON
16151,PO:0003023,root nodule,skos:closeMatch,ENVO:01000164,root nodule,semapv:LexicalMatching,oaklib,0.5,rdfs:label,rdfs:label,nodule root,PO,ENVO
16338,PO:0009047,stem,skos:closeMatch,CHMO:0000086,scanning transmission electron microscopy,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasExactSynonym,stem,PO,CHMO
16339,PO:0025034,leaf,skos:closeMatch,CHMO:0000284,laser-enhanced atomic fluorescence spectroscopy,semapv:LexicalMatching,oaklib,0.5,rdfs:label,oio:hasExactSynonym,leaf,PO,CHMO


In [30]:
predictable_df = pd.concat([mixs_nmdc_subj_frame, reversed_obj_frame, enriched_for_pvs_frame], axis=0, ignore_index=True)

In [31]:
# Remove duplicate rows and force a new copy
unique_df = predictable_df.drop_duplicates().copy()

In [32]:
unique_df['subject_prefix'].value_counts()

subject_prefix
nmdc    2876
ENVO     884
mixs     352
PO       116
Name: count, dtype: int64

In [33]:
unique_df['object_prefix'].value_counts()

object_prefix
nmdc         991
GOLDVOCAB    463
sweet        454
GOLDTERMS    311
NCIT         299
            ... 
FLOPO          1
CHEMINF        1
PO             1
PCO            1
ECOSIM         1
Name: count, Length: 74, dtype: int64

In [34]:
# Write the combined DataFrame to a TSV
unique_df.to_csv(output_tsv, sep='\t', index=False)

print(f"Combined TSV written to {output_tsv}")


Combined TSV written to lexmatch-combined.tsv


In [35]:
# Read the file and create a set of lowercase strings
with open(for_lexmatch_txt, 'r') as f:
    requested_ontologies = {line.strip().lower() for line in f if len(line.strip().lower()) > 0}

In [36]:
# Get DataFrame prefixes as a set
hit_prefixes = set(unique_df['object_prefix'].str.lower())

In [37]:
requests_only = list(requested_ontologies - hit_prefixes)
requests_only.sort()

In [38]:
requests_only

['cob', 'ecto', 'mop', 'omo', 'schema-dot-org', 'sweetall']

In [39]:
hits_only = list(hit_prefixes - requested_ontologies )
hits_only.sort()

In [40]:
hits_only

['apollo_sv',
 'aro',
 'bao',
 'bfo',
 'bspo',
 'bto',
 'cdao',
 'chebi',
 'cheminf',
 'dce',
 'dcterms',
 'ecosimconcept',
 'eupath',
 'exo',
 'faldo',
 'fbbi',
 'flopo',
 'foodon',
 'geno',
 'go',
 'goldvocab',
 'ino',
 'linkml',
 'maxo',
 'mesh',
 'mi',
 'mixs',
 'mmo',
 'mondo',
 'mp',
 'ncbitaxon',
 'ncit',
 'ncro',
 'nmdc',
 'obiws',
 'obo',
 'ogg',
 'ogms',
 'oio',
 'omp',
 'omrse',
 'pco',
 'pr',
 'prov',
 'pw',
 'ro',
 'ror',
 'schema',
 'skos',
 'so',
 'stato',
 'sweet',
 'to',
 'uberon',
 'uo']